In [ ]:
import argparse
import copy
import json
import os
import sys

try:
    import apex
except:
    print("No APEX!")
import numpy as np
import torch
from torch import nn
import yaml
from det3d import torchie
from det3d.datasets import build_dataloader, build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    batch_processor,
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
    example_to_device,
)
from det3d.torchie.trainer import load_checkpoint
import pickle 
import time 
from matplotlib import pyplot as plt 
from det3d.torchie.parallel import collate, collate_kitti
from torch.utils.data import DataLoader
import matplotlib.cm as cm
import subprocess
import cv2
from tools.demo_utils import visual 
from collections import defaultdict
from det3d.torchie.trainer.utils import all_gather, synchronize
from pathlib import PosixPath
import glob

In [ ]:
def convert_box(info):
    boxes =  info["gt_boxes"].astype(np.float32)
    names = info["gt_names"]

    assert len(boxes) == len(names)

    detection = {}

    detection['box3d_lidar'] = boxes

    # dummy value 
    detection['label_preds'] = np.zeros(len(boxes)) 
    detection['scores'] = np.ones(len(boxes))

    return detection

In [ ]:
cfg = Config.fromfile('configs/nusc/pp/nusc_centerpoint_pp_02voxel_two_pfn_10sweep_demo_mini.py')

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)

dataset = build_dataset(cfg.data.val)

data_loader = DataLoader(
    dataset,
    batch_size=1,
    sampler=None,
    shuffle=False,
    num_workers=8,
    collate_fn=collate_kitti,
    pin_memory=False,
)


In [ ]:
checkpoint = load_checkpoint(model, './latest.pth', map_location="cpu")
model.eval()

model = model.cuda()

gpu_device = torch.device("cuda")
cpu_device = torch.device("cpu")
points_list = [] 
gt_annos = [] 
detections = {}
detections_for_draw = []
points_list = []
token_list = []

# Inference on nuScenes Mini dataset

In [ ]:
for i, data_batch in enumerate(data_loader):
    token = data_batch['metadata'][0]['token']
    token_list.append(token)
    
    # save points data for tensorrt
    data_batch["points"].cpu().numpy()[:,1:].astype(np.float32).tofile( \
                      "./tensorrt/data/centerpoint/points/%s.bin"%token)
    
    # points_list for visulize
    points = data_batch['points'][:, 1:4].cpu().numpy()
    points_list.append(points.T)
    with torch.no_grad():
        outputs = batch_processor(
            model, data_batch, train_mode=False, local_rank=0
        )
    info = dataset._nusc_infos[i]
    gt_annos.append(convert_box(info))
    
    for output in outputs:
        token = output["metadata"]["token"]
        for k, v in output.items():
            if k not in [
                "metadata",
            ]:
                output[k] = v.to(cpu_device)
        detections_for_draw.append(output)
        detections.update(
            {token: output,}
        )

all_predictions = all_gather(detections)

predictions = {}
for p in all_predictions:
    predictions.update(p)

result_dict, _ = dataset.evaluation(copy.deepcopy(predictions), output_dir="./", testset=False)

if result_dict is not None:
    for k, v in result_dict["results"].items():
        print(f"Evaluation {k}: {v}")

# Visualize Pytorch Results

In [ ]:
print('Done model inference. Please wait a minute, the matplotlib is a little slow...')

vis_num = 10
draw_num = min(vis_num, len(points_list))
for i in range(draw_num):
    visual(points_list[i], gt_annos[i], detections_for_draw[i], i, save_path="demo/torch_demo")
    print("Rendered Image {}".format(i))

# Evaluete TensorRT Result
1. copy the ./tensorrt/data/centerpoint/points to <TensorRT root directory\>/data/centerpoint
2. run the <TensorRT root directory\>/bin/centerpoint to get the tensorrt outputs.
3. copy the <TensorRT root directory\>/data/centerpoint back the CenterPoint/tensorrt/data
4. run the following python code to do evaluation and visualiza tensorrt result

In [ ]:
def read_trt_result(path):
    token = path.split("/")[-1].split(".")[0]
    trt_pred = {} 
    with open(path) as f:
        trt_res = f.readlines()

    boxs = []
    box3d = []
    score = []
    cls = []
    for line in trt_res:
        box3d += [np.array([float(it) for it in line.strip().split(" ")[:9]])]
        score += [np.array([float(line.strip().split(" ")[-2])])]
        cls += [np.array([int(line.strip().split(" ")[-1])])]

    trt_pred["box3d_lidar"] = torch.from_numpy(np.array(box3d))
    trt_pred["scores"] = torch.from_numpy(np.array(score))
    trt_pred["label_preds"] = torch.from_numpy(np.array(cls,np.int32))
    trt_pred["metadata"] = {}

    trt_pred["metadata"]["num_point_features"] = 5
    trt_pred["metadata"]["token"] = token
    
    return trt_pred, token

In [ ]:
points_dict = {}
points_list = []
gt_annos_dict =  {}
for i, data_batch in enumerate(data_loader):
    token = data_batch['metadata'][0]['token']
    points = data_batch['points'][:, 1:4].cpu().numpy()
    points_dict[token] = points.T
    
    info = dataset._nusc_infos[i]
    gt_annos_dict[token] = convert_box(info)

In [ ]:
trt_pred = {}
detections = {}
detections_for_draw = []
gt_annos = []
res_path_list = glob.glob("./tensorrt/data/centerpoint/results/*.txt")
output_dict = {}

for path in res_path_list:
    output, token = read_trt_result(path)
    output_dict[token] = output

for token in token_list:    
    points_list.append(points_dict[token])
    gt_annos.append(gt_annos_dict[token])
    output = output_dict[token]
    for k, v in output.items():
        if k not in [
            "metadata",
        ]:
            output[k] = v
    detections_for_draw.append(output)
    detections.update(
        {token: output,}
    )
all_predictions = all_gather(detections)

predictions = {}
for p in all_predictions:
    predictions.update(p)

result_dict, _ = dataset.evaluation(copy.deepcopy(predictions), output_dir="./", testset=False)

if result_dict is not None:
    for k, v in result_dict["results"].items():
        print(f"Evaluation {k}: {v}")

# Visualize TensorRT Result

In [ ]:
print('Done model inference. Please wait a minute, the matplotlib is a little slow...')
vis_num = 10
draw_num = min(vis_num, len(points_list))
for i in range(draw_num):
    visual(points_list[i], gt_annos[i], detections_for_draw[i], i, save_path="demo/trt_demo")
    print("Rendered Image {}".format(i))